In [4]:
%load_ext lab_black

Step: Load data

In [5]:
import pandas as pd
import numpy as np
from azureml.core import Workspace
import os
from pyprojroot import here
from random import shuffle

path_data = here("./data")
os.chdir(path_data)

data_tournament_app = pd.read_excel("data_tournament_app.xlsx")

In [6]:
data_tournament_app.sort_values(by="points", ascending=False).reset_index(drop=True)

,player name,points
0,EEB 7,120
1,EEB 17,110
2,EEB 2,110
3,EEB 18,105
4,EEB 1,100
5,EEB 16,100
6,EEB 20,98
7,EEB 19,95
8,EEB 3,90
9,EEB 4,75


In [10]:
def assign_players(df, n_seeds):
    df = df.sort_values(by="points", ascending=False).reset_index(drop=True)
    df["seed"] = np.nan
    df.loc[: n_seeds - 1, "seed"] = range(1, n_seeds + 1)

    byes = []
    if df.shape[0] % 2 != 0:
        for seed in range(1, n_seeds + 1):
            byes.append(
                {
                    "Player 1": df[df["seed"] == seed]["player name"].values[0],
                    "Player 2": None,
                    "Seed": seed,
                }
            )
            df = df[df["seed"] != seed]
            if df.shape[0] % 2 == 0:
                break

    seeds_df = df[df["seed"].notna()]
    non_seeds_df = df[df["seed"].isna()]

    seed_matchups = []
    for index, seed in seeds_df.iterrows():
        if len(non_seeds_df) > 0:
            matchup = non_seeds_df.iloc[0]
            non_seeds_df = non_seeds_df.iloc[1:]
            seed_matchups.append(
                {
                    "Player 1": seed["player name"],
                    "Player 2": matchup["player name"],
                    "Seed": seed["seed"],
                }
            )

    non_seeds = list(non_seeds_df["player name"])
    shuffle(non_seeds)
    non_seed_matchups = [
        {"Player 1": non_seeds[i], "Player 2": non_seeds[i + 1], "Seed": np.nan}
        for i in range(0, len(non_seeds), 2)
    ]

    byes_df = pd.DataFrame(byes)
    seed_matchups_df = pd.DataFrame(seed_matchups)
    non_seed_matchups_df = pd.DataFrame(non_seed_matchups)

    if n_seeds == 1:
        matchups_df = pd.concat(
            [byes_df, seed_matchups_df, non_seed_matchups_df]
        ).reset_index(drop=True)
    elif n_seeds >= 2:
        first_seed = (
            byes_df[byes_df["Seed"] == 1]
            if not byes_df.empty and 1 in byes_df["Seed"].values
            else seed_matchups_df[seed_matchups_df["Seed"] == 1]
        )
        second_seed = (
            byes_df[byes_df["Seed"] == 2]
            if not byes_df.empty and 2 in byes_df["Seed"].values
            else seed_matchups_df[seed_matchups_df["Seed"] == 2]
        )
        third_seed = (
            byes_df[byes_df["Seed"] == 3]
            if not byes_df.empty and 3 in byes_df["Seed"].values
            else seed_matchups_df[seed_matchups_df["Seed"] == 3]
            if n_seeds >= 3
            else pd.DataFrame()
        )
        fourth_seed = (
            byes_df[byes_df["Seed"] == 4]
            if not byes_df.empty and 4 in byes_df["Seed"].values
            else seed_matchups_df[seed_matchups_df["Seed"] == 4]
            if n_seeds >= 4
            else pd.DataFrame()
        )

        non_seed_half = len(non_seed_matchups_df) // 2
        first_half = non_seed_matchups_df.iloc[:non_seed_half]
        second_half = non_seed_matchups_df.iloc[non_seed_half:]

        matchups_df = pd.concat(
            [first_seed, first_half, third_seed, fourth_seed, second_half, second_seed]
        ).reset_index(drop=True)

    return matchups_df

In [9]:
assign_players(df=data_tournament_app.head(13), n_seeds=1)

,Player 1,Player 2,Seed
0,EEB 7,None,1.0
1,EEB 13,EEB 8,NaN
2,EEB 5,EEB 3,NaN
3,EEB 2,EEB 1,NaN
4,EEB 12,EEB 11,NaN
5,EEB 10,EEB 4,NaN
6,EEB 9,EEB 6,NaN
